**<h3>In this notebook, we covered the fine tuning of</h3>**
1. YOLOv9-E (Largest Model - 57.3M Params)

with changing
- Optimizer
- Learning Rate
- Batch Size
- Freezing Layer
- Image Size

Before that, download all the libraries required and import it.

In [ ]:
!pip install ultralytics

In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
from ultralytics import settings
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import os
import yaml
import torch
import cv2
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


**<h4>Section 1: Data Collection</h4>**
Image data will be collected from Roboflow. We selected the version 3 of the data which contains a total of 5555 images, from
- **Training Set: 5102**
- **Validation Set: 432**
- **Test Set: 21**

with format YOLOv9 which all images are labelled.

References Link: https://universe.roboflow.com/plat-kendaraan/vehicle-and-license-plate/dataset/3#

In [ ]:
############################
#Coded by Teo Teck Wan, Lee Yong Jin, Chen Si Yang, Yvonne Eng Xin Yee
############################
rf = Roboflow(api_key="8rCFSuNNOBXXSFKJejv6")
project = rf.workspace("plat-kendaraan").project("vehicle-and-license-plate")
version = project.version(3)
dataset = version.download("yolov9")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to vehicle-and-license-plate-3 in yolov9:: 100%|██████████| 11122/11122 [00:01<00:00, 5917.65it/s]


**<h4>Section 2: Data Preprocessing</h4>**
Below is the yaml file, which recorded the metadata of the dataset.
- data.yaml
```
train: ../train/images
val: ../valid/images
test: ../test/images
nc: 4
names: ['License_Plate', 'cars', 'motorcyle', 'truck']
```
Since our mission is to detect the car plate only, hence nc will be '1' and names only be 'License Plate'. And the path for train, val, and test are changing according to colab path.

---

Even though all the images are labelled, but half of them are not in the YOLOv9 format.
```
#YOLOv9 Annotation Format:
class, x_center, y_center, width, height
```
But half of the labels are in the format of
```
class, x1, y1, x2, y2, x3, y3, x4, y4, x1, y1
```
Hence, this will be converted into the format of YOLOv9.

In [ ]:
############################
#Coded by Teo Teck Wan, Lee Yong Jin, Chen Si Yang, Yvonne Eng Xin Yee
############################
# This function is to auto update the updated annotation in YOLO format
def autoUpdate(sourcePath, annotationsCombo):
    # Open the file in write mode
    with open(sourcePath, 'w') as file2:
        ann = annotationsCombo

        for i, annotation in enumerate(ann):
            # Convert each annotation to a string
            line = ' '.join(map(str, annotation))

            # Write the line to the file
            file2.write(line)

            # If it's not the last line, add a newline character
            if i != len(annotations) - 1:
                file2.write('\n')

    return ann

In [ ]:
############################
#Coded by Teo Teck Wan, Lee Yong Jin, Chen Si Yang, Yvonne Eng Xin Yee
############################
# Update the annotations in all the labels' files
labelFolders = ['train', 'valid', 'test']

for labelFolder in labelFolders:
      labelDirectory = f'/content/vehicle-and-license-plate-3/{labelFolder}/labels'
      for fileName in os.listdir(labelDirectory):
          filePath = os.path.join(labelDirectory, fileName)

          # Annotations in YOLO format (class, x_center, y_center, width, height)
          annotations = []

          # Open the file and read its contents
          with open(filePath, 'r') as file:
              for line in file:
                  # Split the line into a list of numbers
                  numbers = [float(num) for num in line.strip().split()]

                  # For the file that are not in YOLO format yet [class (x1 y1) (x2 y2) (x3 y3) (x4 y4) (x1 y1)]
                  # class 0 = license plate
                  if (numbers[0] == 0 and len(numbers) > 5):
                      c = 0
                      all_x = [numbers[idx] for idx in range(1, len(numbers)-2, 2)]
                      all_y = [numbers[idx] for idx in range(2, len(numbers)-2, 2)]

                      x1 = min(all_x)
                      y1 = min(all_y)
                      x2 = max(all_x)
                      y2 = max(all_y)

                      x_center = (x1 + x2) / 2
                      y_center = (y1 + y2) / 2
                      width = x2 - x1
                      height = y2 - y1

                      annotations.append([c, x_center, y_center, width, height])

                  # For the file that already in the YOLO format [class, x_center, y_center, width, height]
                  if (numbers[0] == 0 and len(numbers) < 6):
                      numbers[0] = 0
                      annotations.append(numbers)

              annotations = autoUpdate(filePath, annotations)

In [ ]:
############################
#Coded by Teo Teck Wan, Lee Yong Jin, Chen Si Yang, Yvonne Eng Xin Yee
############################
# The path of YAML file
yamlFilePath = '/content/vehicle-and-license-plate-3/data.yaml'

# Load the existing data from the YAML file
with open(yamlFilePath, 'r') as file:
    yamlData = yaml.safe_load(file)

# Update the values
yamlData['train'] = '/content/vehicle-and-license-plate-3/train/images'
yamlData['val'] = '/content/vehicle-and-license-plate-3/valid/images'
yamlData['test'] = '/content/vehicle-and-license-plate-3/test/images'
yamlData['nc'] = 1  # Updating the number of classes to 1
yamlData['names'] = ['License_Plate']  # Updating the names list to only include 'License_Plate'

# Save the updated data back into the YAML file
with open(yamlFilePath, 'w') as file:
    yaml.safe_dump(yamlData, file, default_flow_style=False, sort_keys=False)

print(f"Updated YAML file saved at {yamlFilePath}")

Updated YAML file saved at /content/vehicle-and-license-plate-3/data.yaml


In [ ]:
############################
#Coded by Teo Teck Wan, Lee Yong Jin, Chen Si Yang, Yvonne Eng Xin Yee
############################
# Update the library model's path from setting.yaml
# View all settings
print(f'Before update: {settings}')

# Update dataset path
settings['datasets_dir'] = '/content/vehicle-and-license-plate-3'
print(f'After update: {settings}')


Before update: {'settings_version': '0.0.4', 'datasets_dir': '/content/datasets', 'weights_dir': 'weights', 'runs_dir': 'runs', 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
After update: {'settings_version': '0.0.4', 'datasets_dir': '/content/vehicle-and-license-plate-3', 'weights_dir': 'weights', 'runs_dir': 'runs', 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}


**<h4>Section 3: Data Training</h4>**
We fine tuning the
1. YOLOv9-E (Largest Model - 57.3M Params)

Firstly, we loaded the original model (1) and start perform the custom training.
During the training, it actually takes the train folder to train and valid folder to validate the data.

```
Summary sections:
(yolov9_b16.pt)
100 Epochs, 16 Batch, AdamW Optimizer, 0.002 LR
-> mAP50 = 0.994, mAP50-95 = 0.895, Training Hours = 3.801

(yolov9_b32.pt)
100 Epochs, 32 Batch, AdamW Optimizer, 0.002 LR
-> mAP50 = 0.994, mAP50-95 = 0.902, Training Hours = 3.644

(yolov9_b32_f10.pt)
100 Epochs, 32 Batch, Freezing 10 Layer (All Backbone), AdamW Optimizer, 0.002 LR
-> mAP50 = 0.994, mAP50-95 = 0.936, Training Hours = 2.886

(yolo9_b16_f10_sgd_i720.pt)
100 Epochs, 16 Batch, Freezing 10 Layer, SGD Optimizer, 0.01 LR, 720px Image Size
-> mAP50 = 0.994, mAP50-95 = 0.948, Training Hours = 3.801

Noted: the results folder mounted from Drive stored the trained best weight (that trained previously)
so that no required to train again since training hours is more than 2.5.
```




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
############################
# For the YOLO()
# Reference: https://docs.ultralytics.com/quickstart/                 (Ultralytics Library)
############################
# Build a YOLOv9e model from pretrained weight
model = YOLO('yolov9e.pt')

# Display model information (optional)
model.info()

100%|██████████| 112M/112M [00:00<00:00, 404MB/s] 


YOLOv9e summary: 1225 layers, 58206592 parameters, 0 gradients, 193.0 GFLOPs


(1225, 58206592, 0, 193.01565439999996)

In [ ]:
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Silence()
      (1): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (3): RepNCSPELAN4(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Sequential(
          (0): RepCSP(
            (cv1): Conv(
              (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=

In [ ]:
############################
# For the .train()
# Reference: https://docs.ultralytics.com/quickstart/                 (Ultralytics Library)
############################
# This will be the yolov9_b16.pt after training
results_b16 = model.train(data=yamlFilePath, epochs=100, imgsz=640, batch=16) # output misclick, but this model is saved

In [ ]:
# This will be yolov9_b32.pt after training
results_b32 = model.train(data=yamlFilePath, epochs=100, imgsz=640, batch=32) # output misclick, but this model is saved

In [ ]:
# This will be yolov9_b32_f10.pt after training
results_b32_f10 = model.train(data=yamlFilePath, epochs=100, imgsz=640, batch=32, freeze=10)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov9e.pt, data=/content/drive/MyDrive/DeepLearning/dataset1/data.yaml, epochs=100, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False

train: Scanning /content/drive/MyDrive/DeepLearning/dataset1/train/labels.cache... 5102 images, 4 backgrounds, 0 corrupt: 100%|██████████| 5106/5106 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/DeepLearning/dataset1/valid/labels.cache... 432 images, 0 backgrounds, 0 corrupt: 100%|██████████| 432/432 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 298 weight(decay=0.0), 310 weight(decay=0.0005), 309 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      25.7G       0.31     0.1674     0.8368         20        640: 100%|██████████| 160/160 [01:36<00:00,  1.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:04<00:00,  1.69it/s]

                   all        432        463      0.991      0.988      0.994      0.934



100 epochs completed in 2.886 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 117.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 117.3MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv9e summary (fused): 687 layers, 57377171 parameters, 0 gradients, 189.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.38it/s]


                   all        432        463      0.991      0.987      0.994      0.936
Speed: 0.1ms preprocess, 7.2ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train2


In [ ]:
# This will be yolov9_b16_f10_sgd_i720.pt after training
results_b16_f10_sgd_i720 = model.train(data=yamlFilePath, epochs=100, imgsz=720, batch=16, freeze=10, lr0=0.01, optimizer='SGD', momentum=0.9)

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov9e.pt, data=/content/vehicle-and-license-plate-3/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

train: Scanning /content/vehicle-and-license-plate-3/train/labels.cache... 5102 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5102/5102 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
val: Scanning /content/vehicle-and-license-plate-3/valid/labels.cache... 432 images, 0 backgrounds, 0 corrupt: 100%|██████████| 432/432 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 298 weight(decay=0.0), 310 weight(decay=0.0005), 309 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 736 train, 736 val
Using 8 dataloader workers
Logging results to runs/detect/train4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      19.8G      0.277     0.1387     0.8251         15        736: 100%|██████████| 319/319 [02:10<00:00,  2.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:04<00:00,  2.87it/s]

                   all        432        463      0.995      0.994      0.994      0.947



100 epochs completed in 3.806 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 117.3MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 117.3MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv9e summary (fused): 687 layers, 57377171 parameters, 0 gradients, 189.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:06<00:00,  2.32it/s]


                   all        432        463      0.991      0.996      0.994      0.948
Speed: 0.2ms preprocess, 9.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train4


After we trained the model, the function will automatically save the results.

In the results folder, which included many of things, however this focused on
```
1. Weighted model for best and last (best.pt)
2. Box Loss, standard classification Cross Entropy Loss (cls), Distribution Focal Loss (dfl) (3 Images)
3. Precision and Recall metrics (Image)
4. mAP50 & mAP50-95 metrics for best performance (Image)
```



In [ ]:
############################
#Coded by Teo Teck Wan, Lee Yong Jin, Chen Si Yang, Yvonne Eng Xin Yee
############################
# Save results folder in local for next time the model no need to re-train (For references)
import zipfile

with zipfile.ZipFile('result-model.zip', 'w') as zip_file:
    # Add all files in the 'runs/detect/' directory to the zip file
    for root, dirs, files in os.walk('runs/detect/'):
        for file_name in files:
            zip_file.write(os.path.join(root, file_name))

In [ ]:
WEIGHTDIR = '/content/drive/MyDrive/DeepLearning/Assignment 2/results'

# Load model
# Original YOLOv9 model = model

# YOLOv9 - epoch: 100 - batch: 16, Optimizer: AdamW, lr=0.002, momentum=0.9
model_b16 = YOLO(os.path.join(WEIGHTDIR, 'runs-b16/weights/yolov9_b16.pt'))

# YOLOv9 - epoch: 100 - batch: 32, Optimizer: AdamW, lr=0.002, momentum=0.9
model_b32 = YOLO(os.path.join(WEIGHTDIR, 'runs-b32/weights/yolov9_b32.pt'))

# YOLOv9 - epoch: 100 - batch: 32, freeze: 10, Optimizer: AdamW, lr=0.002, momentum=0.9
model_b32_f10 = YOLO(os.path.join(WEIGHTDIR, 'runs-b32f10/weights/yolov9_b32_f10.pt'))

# YOLOv9 - epoch: 100 - batch: 16, freeze: 10, Optimizer: SGD, lr=0.01, momentum=0.9, Image resize to 720px
model_b16_f10_sgd = YOLO(os.path.join(WEIGHTDIR, 'runs-b16f10sgd/weights/yolo9_b16_f10_sgd_i720.pt'))

**<h4>Section 4: Data Evaluation</h4>**
Since we mounted the Google Drive (as the previous fine-tuned model was saved in local), we will validate the model using dataset's test folder (which the data will not be seen before for the trained model).

```
Summary sections:
(yolov9_b16.pt)
-> mAP50 = 0.995, mAP50-95 = 0.920

(yolov9_b32.pt)
-> mAP50 = 0.995, mAP50-95 = 0.924

(yolov9_b32_f10.pt)
-> mAP50 = 0.995, mAP50-95 = 0.960

(yolo9_b16_f10_sgd_i720.pt)
-> mAP50 = 0.977, mAP50-95 = 0.960
```



In [ ]:
############################
# For the .val()
# Reference: https://docs.ultralytics.com/quickstart/                 (Ultralytics Library)
############################
rslt_b16 = model_b16.val(data=yamlFilePath, split='test')

Ultralytics YOLOv8.2.0 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv9e summary (fused): 687 layers, 57377171 parameters, 0 gradients, 189.1 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 44.6MB/s]
val: Scanning /content/vehicle-and-license-plate-3/test/labels... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<00:00, 709.87it/s]

val: New cache created: /content/vehicle-and-license-plate-3/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.93s/it]


                   all         21         23      0.999          1      0.995       0.92
Speed: 4.1ms preprocess, 57.2ms inference, 0.0ms loss, 122.0ms postprocess per image
Results saved to runs/detect/val


In [ ]:
rslt_b32 = model_b32.val(data=yamlFilePath, split='test')

Ultralytics YOLOv8.2.0 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv9e summary (fused): 687 layers, 57377171 parameters, 0 gradients, 189.1 GFLOPs


val: Scanning /content/vehicle-and-license-plate-3/test/labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.46it/s]


                   all         21         23      0.997          1      0.995      0.924
Speed: 3.3ms preprocess, 42.1ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/val2


In [ ]:
rslt_b32_f10 = model_b32_f10.val(data=yamlFilePath, split='test')

Ultralytics YOLOv8.2.0 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv9e summary (fused): 687 layers, 57377171 parameters, 0 gradients, 189.1 GFLOPs


val: Scanning /content/vehicle-and-license-plate-3/test/labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.58it/s]


                   all         21         23          1      0.998      0.995       0.96
Speed: 3.5ms preprocess, 41.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val3


In [ ]:
rslt_b32_f10_sgd = model_b16_f10_sgd.val(data=yamlFilePath, split='test')

Ultralytics YOLOv8.2.0 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv9e summary (fused): 687 layers, 57377171 parameters, 0 gradients, 189.1 GFLOPs


val: Scanning /content/vehicle-and-license-plate-3/test/labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.39it/s]


                   all         21         23      0.998      0.957      0.977       0.96
Speed: 4.6ms preprocess, 55.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val4


**<h4>Section 5: Inferences</h4>**
After that, we tried to input the trained models to test whether it is detectable to the car plate by image and video (Not from the source database that we used to train).

In [ ]:
############################
# For all the .predict() in this section
# Reference: https://docs.ultralytics.com/quickstart/                 (Ultralytics Library)
############################

# Model b16
model_b16.predict('/content/drive/MyDrive/DeepLearning/Assignment 2/car image 3.jpg', device=0, save=True)
# Result will be saved (path can be taken from the output)


image 1/1 /content/drive/MyDrive/DeepLearning/Assignment 2/car image 3.jpg: 640x640 (no detections), 32.4ms
Speed: 3.3ms preprocess, 32.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'License_Plate'}
 obb: None
 orig_img: array([[[ 21,  25,  20],
         [ 19,  23,  18],
         [ 23,  27,  22],
         ...,
         [ 53,  88,  61],
         [ 41,  73,  48],
         [ 31,  63,  38]],
 
        [[ 17,  21,  16],
         [ 16,  20,  15],
         [ 17,  21,  16],
         ...,
         [ 54,  89,  62],
         [ 40,  72,  47],
         [ 32,  64,  39]],
 
        [[ 12,  16,  11],
         [ 12,  16,  11],
         [ 12,  16,  11],
         ...,
         [ 50,  85,  58],
         [ 42,  74,  49],
         [ 44,  74,  49]],
 
        ...,
 
        [[218, 218, 218],
         [219, 219, 219],
         [222, 222, 222],
         ...,
         [153, 161, 161],
         [158, 163, 164],
         [161, 166, 167]],
 
        [[211, 211, 211],
         [211, 211, 211],
         [214, 214, 214],
         ...,
         [1

In [ ]:
# Model b32
model_b32.predict('/content/drive/MyDrive/DeepLearning/Assignment 2/car image 3.jpg', device=0, save=True)


image 1/1 /content/drive/MyDrive/DeepLearning/Assignment 2/car image 3.jpg: 640x640 (no detections), 35.0ms
Speed: 3.2ms preprocess, 35.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'License_Plate'}
 obb: None
 orig_img: array([[[ 21,  25,  20],
         [ 19,  23,  18],
         [ 23,  27,  22],
         ...,
         [ 53,  88,  61],
         [ 41,  73,  48],
         [ 31,  63,  38]],
 
        [[ 17,  21,  16],
         [ 16,  20,  15],
         [ 17,  21,  16],
         ...,
         [ 54,  89,  62],
         [ 40,  72,  47],
         [ 32,  64,  39]],
 
        [[ 12,  16,  11],
         [ 12,  16,  11],
         [ 12,  16,  11],
         ...,
         [ 50,  85,  58],
         [ 42,  74,  49],
         [ 44,  74,  49]],
 
        ...,
 
        [[218, 218, 218],
         [219, 219, 219],
         [222, 222, 222],
         ...,
         [153, 161, 161],
         [158, 163, 164],
         [161, 166, 167]],
 
        [[211, 211, 211],
         [211, 211, 211],
         [214, 214, 214],
         ...,
         [1

In [ ]:
# Model b32_f10
model_b32_f10.predict('/content/drive/MyDrive/DeepLearning/Assignment 2/car image 3.jpg', device=0, save=True)


image 1/1 /content/drive/MyDrive/DeepLearning/Assignment 2/car image 3.jpg: 640x640 3 License_Plates, 32.8ms
Speed: 3.1ms preprocess, 32.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict3


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'License_Plate'}
 obb: None
 orig_img: array([[[ 21,  25,  20],
         [ 19,  23,  18],
         [ 23,  27,  22],
         ...,
         [ 53,  88,  61],
         [ 41,  73,  48],
         [ 31,  63,  38]],
 
        [[ 17,  21,  16],
         [ 16,  20,  15],
         [ 17,  21,  16],
         ...,
         [ 54,  89,  62],
         [ 40,  72,  47],
         [ 32,  64,  39]],
 
        [[ 12,  16,  11],
         [ 12,  16,  11],
         [ 12,  16,  11],
         ...,
         [ 50,  85,  58],
         [ 42,  74,  49],
         [ 44,  74,  49]],
 
        ...,
 
        [[218, 218, 218],
         [219, 219, 219],
         [222, 222, 222],
         ...,
         [153, 161, 161],
         [158, 163, 164],
         [161, 166, 167]],
 
        [[211, 211, 211],
         [211, 211, 211],
         [214, 214, 214],
         ...,
         [1

In [ ]:
# Model b16_f10_sgd_i720
model_b16_f10_sgd.predict('/content/drive/MyDrive/DeepLearning/Assignment 2/car image 3.jpg', device=0, save=True)


image 1/1 /content/drive/MyDrive/DeepLearning/Assignment 2/car image 3.jpg: 736x736 2 License_Plates, 33.4ms
Speed: 4.1ms preprocess, 33.4ms inference, 1.3ms postprocess per image at shape (1, 3, 736, 736)
Results saved to runs/detect/predict4


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'License_Plate'}
 obb: None
 orig_img: array([[[ 21,  25,  20],
         [ 19,  23,  18],
         [ 23,  27,  22],
         ...,
         [ 53,  88,  61],
         [ 41,  73,  48],
         [ 31,  63,  38]],
 
        [[ 17,  21,  16],
         [ 16,  20,  15],
         [ 17,  21,  16],
         ...,
         [ 54,  89,  62],
         [ 40,  72,  47],
         [ 32,  64,  39]],
 
        [[ 12,  16,  11],
         [ 12,  16,  11],
         [ 12,  16,  11],
         ...,
         [ 50,  85,  58],
         [ 42,  74,  49],
         [ 44,  74,  49]],
 
        ...,
 
        [[218, 218, 218],
         [219, 219, 219],
         [222, 222, 222],
         ...,
         [153, 161, 161],
         [158, 163, 164],
         [161, 166, 167]],
 
        [[211, 211, 211],
         [211, 211, 211],
         [214, 214, 214],
         ...,
         [1

In [ ]:
# test on video
model_b16_f10_sgd.predict('/content/drive/MyDrive/DeepLearning/Assignment 2/a.mp4', device=0, save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/631) /content/drive/MyDrive/DeepLearning/Assignment 2/a.mp4: 480x736 1 License_Plate, 209.3ms
video 1/1 (frame 2/631) /content/drive/MyDrive/DeepLearning/Assignment 2/a.mp4: 480x736 1 License_Plate, 32.6ms
video 1/1 (frame 3/631) /content/drive/MyDrive/DeepLearning/Assignment 2/a.mp4: 480x736 1 License_Plate, 33.3ms
video 1/1 (frame 4/631) /content/drive/MyDrive/DeepLearning/Assignment 2/a.mp4: 480x736 1 License_Plate, 38.2ms
video 1

All speed (FPS) result will be done here for every models.<br>
Here is the summary.
```
RCNN-1 (Yong Jin):
Average Inference FPS: 24.07
End-to-End FPS: 22.40

RCNN-2 (Yvonne):
Average Inference FPS: 24.22
End-to-End FPS: 22.47

Yolov8:
Average Inference FPS: 56.98
End-to-End FPS: 47.35

Yolov9:
Average Inference FPS: 20.33
End-to-End FPS: 18.98
```



In [ ]:
############################
#Coded by Teo Teck Wan, Lee Yong Jin, Chen Si Yang, Yvonne Eng Xin Yee
############################
# Load fine-tuned RCNN model (Model 1 and Model 2)
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch

weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn(weights=weights)
model2 = fasterrcnn_resnet50_fpn(weights=weights)

in_features = model.roi_heads.box_predictor.cls_score.in_features
num_classes = 2
model.roi_heads.box_predictor = FastRCNNPredictor(in_channels=in_features, num_classes=num_classes)
model2.roi_heads.box_predictor = FastRCNNPredictor(in_channels=in_features, num_classes=num_classes)
torch.save(model2.state_dict(), '/content/drive/MyDrive/DeepLearning/Assignment 2/carobjdetec.pt')
torch.save(model2.state_dict(), '/content/drive/MyDrive/DeepLearning/Assignment 2/frcnn_final.pt')

In [ ]:
model.eval()
model2.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
modelv8 = YOLO('/content/drive/MyDrive/DeepLearning/Assignment 2/yolov8n.pt')

WARNING ⚠️ /content/drive/MyDrive/DeepLearning/Assignment 2/yolov8n.pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.0s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



In [ ]:
############################
#Coded by Teo Teck Wan, Lee Yong Jin, Chen Si Yang, Yvonne Eng Xin Yee
############################
def process_video(video_path, model, device='cuda'):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    frame_count = 0
    total_inference_time = 0
    total_preprocessing_time = 0
    total_postprocessing_time = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Start timing preprocessing
        start_preprocessing = time.time()

        # Resize frame to be divisible by 32 (e.g., 640x640)
        height, width = frame.shape[:2]
        new_height = (height // 32) * 32
        new_width = (width // 32) * 32
        frame = cv2.resize(frame, (new_width, new_height))

        # Convert frame to the format expected by the model
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_tensor = torch.from_numpy(img).to(device).float() / 255.0  # Normalize to [0, 1]
        img_tensor = img_tensor.permute(2, 0, 1).unsqueeze(0)  # B C H W

        end_preprocessing = time.time()
        total_preprocessing_time += (end_preprocessing - start_preprocessing)

        # Start timing inference
        start_inference = time.time()
        with torch.no_grad():
            results = model(img_tensor)
        end_inference = time.time()

        total_inference_time += (end_inference - start_inference)

        frame_count += 1

    cap.release()

    if frame_count > 0:
        average_inference_time = total_inference_time / frame_count
        average_total_time = (total_inference_time + total_preprocessing_time) / frame_count
        inference_fps = 1 / average_inference_time
        end_to_end_fps = 1 / average_total_time
    else:
        inference_fps = end_to_end_fps = 0

    print(f"Average Inference FPS: {inference_fps:.2f}")
    print(f"End-to-End FPS: {end_to_end_fps:.2f}")


In [ ]:
process_video('/content/drive/MyDrive/DeepLearning/Assignment 2/a.mp4', model.to("cuda"))

Average Inference FPS: 24.07
End-to-End FPS: 22.40


In [ ]:
process_video('/content/drive/MyDrive/DeepLearning/Assignment 2/a.mp4', model2.to("cuda"))

Average Inference FPS: 24.22
End-to-End FPS: 22.47


In [ ]:
process_video('/content/drive/MyDrive/DeepLearning/Assignment 2/a.mp4', model_b16_f10_sgd)


0: 704x1088 1 License_Plate, 228.4ms
Speed: 1.2ms preprocess, 228.4ms inference, 3.0ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 41.9ms
Speed: 0.0ms preprocess, 41.9ms inference, 2.1ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 41.9ms
Speed: 0.0ms preprocess, 41.9ms inference, 2.1ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 43.0ms
Speed: 0.0ms preprocess, 43.0ms inference, 2.1ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 41.7ms
Speed: 0.0ms preprocess, 41.7ms inference, 2.1ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 41.7ms
Speed: 0.0ms preprocess, 41.7ms inference, 2.1ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 41.8ms
Speed: 0.0ms preprocess, 41.8ms inference, 2.2ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 46.8ms
Speed: 0.0ms pre

In [ ]:
process_video('/content/drive/MyDrive/DeepLearning/Assignment 2/a.mp4', modelv8)


0: 704x1088 1 License_Plate, 61.9ms
Speed: 0.5ms preprocess, 61.9ms inference, 2.3ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 9.7ms
Speed: 0.1ms preprocess, 9.7ms inference, 2.1ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 9.4ms
Speed: 0.0ms preprocess, 9.4ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 9.4ms
Speed: 0.0ms preprocess, 9.4ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 9.3ms
Speed: 0.0ms preprocess, 9.3ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 9.2ms
Speed: 0.0ms preprocess, 9.2ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 10.2ms
Speed: 0.0ms preprocess, 10.2ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1088)

0: 704x1088 1 License_Plate, 9.8ms
Speed: 0.0ms preprocess, 9.8m